<a href="https://colab.research.google.com/github/perkeje/fif_the_statue_cnn/blob/main/Copy_of_FindTheStatue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import tensorflow as tf
from tensorflow import keras

!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py


--2022-08-07 08:26:10--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-08-07 08:26:10 (34.0 MB/s) - ‘helper_functions.py.1’ saved [10246/10246]



In [2]:
from helper_functions import load_and_prep_image,unzip_data,create_tensorboard_callback,plot_loss_curves, compare_historys

In [3]:
tf.version.VERSION

'2.8.2'

In [4]:
for dirpath, dirnames, filenames in os.walk("/content/drive/MyDrive/ZavrssniRad/train"):
  print(f"There are {len(dirnames)} dirs and {len(filenames)} images in our{dirpath}")

There are 9 dirs and 0 images in our/content/drive/MyDrive/ZavrssniRad/train
There are 0 dirs and 51 images in our/content/drive/MyDrive/ZavrssniRad/train/5
There are 0 dirs and 41 images in our/content/drive/MyDrive/ZavrssniRad/train/6
There are 0 dirs and 59 images in our/content/drive/MyDrive/ZavrssniRad/train/8
There are 0 dirs and 37 images in our/content/drive/MyDrive/ZavrssniRad/train/0
There are 0 dirs and 40 images in our/content/drive/MyDrive/ZavrssniRad/train/2
There are 0 dirs and 44 images in our/content/drive/MyDrive/ZavrssniRad/train/3
There are 0 dirs and 45 images in our/content/drive/MyDrive/ZavrssniRad/train/7
There are 0 dirs and 60 images in our/content/drive/MyDrive/ZavrssniRad/train/1
There are 0 dirs and 35 images in our/content/drive/MyDrive/ZavrssniRad/train/4


In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


IMAGE_SHAPE = (256,256)
BATCH_SIZE = 32

# train_dir = "/content/drive/MyDrive/ZavrssniRad/train"
# test_dir = "/content/drive/MyDrive/ZavrssniRad/test"
# valid_dir ="/content/drive/MyDrive/ZavrssniRad/test"
all_dir = "/content/drive/MyDrive/ZavrssniRad/train"
test_dir = "/content/drive/MyDrive/ZavrssniRad/test"

                         


train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

train_data = tf.keras.utils.image_dataset_from_directory(all_dir,
                                              shuffle = True,
                                              image_size=IMAGE_SHAPE,
                                              batch_size=BATCH_SIZE,
                                              label_mode="categorical"
                                              )

test_data = tf.keras.utils.image_dataset_from_directory(test_dir,
                                              shuffle = True,
                                              image_size=IMAGE_SHAPE,
                                              batch_size=BATCH_SIZE,
                                              label_mode="categorical"
)

# print("Training images: ")
# train_data = train_datagen.flow_from_directory(train_dir,
#                                               shuffle = True,
#                                               target_size=IMAGE_SHAPE,
#                                               batch_size=BATCH_SIZE,
#                                               class_mode="categorical")

# print("Testing images: ")
# test_data = test_datagen.flow_from_directory(test_dir,
#                                             shuffle = True,
#                                             target_size=IMAGE_SHAPE,
#                                             batch_size=BATCH_SIZE,
#                                             class_mode="categorical")
# print("Validation images: ")
# valid_data = test_datagen.flow_from_directory(valid_dir,
#                                             shuffle = True,
#                                             target_size=IMAGE_SHAPE,
#                                             batch_size=BATCH_SIZE,
#                                             class_mode="categorical")


Found 412 files belonging to 9 classes.
Found 1018 files belonging to 9 classes.


In [6]:
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential


In [7]:
# import matplotlib.pyplot as plt

# plt.figure(figsize=(10, 10))
# class_names = train_data.class_names
# for images, labels in train_data.take(1):
#     for i in range(32):
#         ax = plt.subplot(6, 6, i + 1)
#         plt.imshow(images[i].numpy().astype("uint8"))
#         plt.axis("off")



In [8]:

data_augmentation = Sequential([
  preprocessing.Rescaling(1./255),
  preprocessing.RandomZoom(0.2)
], name="data_augmentation")



In [9]:
base_model = tf.keras.applications.VGG16(include_top=False)
base_model.trainable=False
inputs = layers.Input(shape=(256,256,3),name="input_layer")
x= data_augmentation(inputs)
x= base_model(x,training=False)
x = layers.GlobalAveragePooling2D(name="global_avg_pool_layer")(x)
outputs = layers.Dense(9,activation="softmax")(x)
model = tf.keras.Model(inputs,outputs)

In [10]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 256, 256, 3)]     0         
                                                                 
 data_augmentation (Sequenti  (None, 256, 256, 3)      0         
 al)                                                             
                                                                 
 vgg16 (Functional)          (None, None, None, 512)   14714688  
                                                                 
 global_avg_pool_layer (Glob  (None, 512)              0         
 alAveragePooling2D)                                             
                                                                 
 dense (Dense)               (None, 9)                 4617      
                                                                 
Total params: 14,719,305
Trainable params: 4,617
Non-trainabl

In [11]:
# checkpoint_path="checkpoint_1"
# checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
#                                                          save_weights_only=True,
#                                                          monitor="val_accuracy",
#                                                          save_best_only=True)

In [12]:
# model.compile(loss="categorical_crossentropy",
#               optimizer=tf.keras.optimizers.Adam(),
#               metrics=["accuracy"])

# history_1 =model.fit(train_data,
#                      epochs=5, # fit for 5 epochs to keep experiments quick
#                      steps_per_epoch= len(train_data),
#                      validation_data=test_data,
#                      validation_steps=int(.1*len(test_data)), # evaluate on smaller portion of test data
#                      callbacks=[checkpoint_callback])

In [13]:
# feature_extraction_model = model.evaluate(test_data)
# feature_extraction_model

In [14]:
#plot_loss_curves(history_1)

In [15]:
base_model.trainable = True
for layer in base_model.layers[:-15]:
  layer.trainable = False
  model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(), # 10x lower learning rate than default
              metrics=['accuracy'])

In [ ]:
fine_tune_epochs = 8

history_fine_tuned = model.fit(train_data,
                     epochs=fine_tune_epochs,
                     validation_data=test_data,
                     validation_steps=int(.1*len(test_data)))
                    #  initial_epoch=history_1.epoch[-1])

Epoch 1/8
13/13 [==============================] - ETA: 0s - loss: 3.0231 - accuracy: 0.1214  

In [ ]:
fine_tuned_model = model.evaluate(test_data)
fine_tuned_model

In [ ]:
plot_loss_curves(history_fine_tuned)

In [ ]:
# compare_historys(history_1,
#                  history_fine_tuned,
#                  5)

In [ ]:
model.evaluate(train_data)

In [ ]:
import matplotlib.pyplot as plt

loyola = load_and_prep_image("/content/drive/MyDrive/ZavrssniRad/train/3/PXL_20220805_182752000.jpg",img_shape=256)
plt.imshow(loyola)

model.predict(tf.expand_dims(loyola,0)).argmax()

In [ ]:
# from keras.models import load_model

# model.save('statue-recognizer.tf')
# !zip -r /content/statue-recognizer.zip /content/statue-recognizer.tf

In [ ]:
# from google.colab import files
# files.download("/content/statue-recognizer.zip")

In [ ]:
# # Convert the model
# converter = tf.lite.TFLiteConverter.from_keras_model(model) # path to the SavedModel directory
# tflite_model = converter.convert()

# # Save the model.
# with open('statue-recognizerv01.tflite', 'wb') as f:
#   f.write(tflite_model)